<a href="https://colab.research.google.com/github/ralphmartynward/ironhack_09_final-project/blob/main/notebooks/googlecolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drive connection

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/My Drive/Admin/School/Online/IronHack/Cours/09 Final Project')

Mounted at /content/drive/


# Install dependencies

In [2]:
!pip install discord
!pip install praw
!pip install transformers
!pip install openai
!pip install instabot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 116.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 139

# Get posts, process & classify, push to Discord server

In [6]:
import discord
import os
import praw
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datetime import datetime, timedelta
import torch
import openai
import nest_asyncio
import asyncio

# Define a function to get credentials
def get_credentials(filename):
    credentials = {}
    with open(filename, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            credentials[key] = value.replace('"', '')
    return credentials

# Load credentials
credentials = get_credentials('cred.txt')

# Initialize Reddit and OpenAI API Key
reddit = praw.Reddit(client_id=credentials["REDDIT_CLIENT_ID"], client_secret=credentials["REDDIT_SECRET"], user_agent="my_user_agent")
openai.api_key = credentials["OPENAI_API_KEY"]

# Initialize model, tokenizer and sentiment analysis pipeline
model_name = "Yueh-Huan/news-category-classification-distilbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, from_tf=True)
nlp = pipeline('sentiment-analysis')

# Category mapping
label_dict = {
'Parents': 0,'Wellness': 1,'Comedy': 3,'Parenting': 2,'Politics': 4,'Black Voices': 5,'Queer Voices': 6,'Entertainment': 7,'Culture & Arts': 8,'Tech': 9,'Religion': 10, 'Style & Beauty': 11, 'Healthy living': 12,'Travel': 13,'Green': 14,'Impact': 15,'Business': 16,'Divorce': 17,'Science': 18,'Sports': 19,'Latino Voices': 20,'World news': 21,'Home & Living': 22,'Media': 23,'U.S. news': 24,'Taste': 25,'Food & Drink': 26,'Weird News': 27,'Style': 28,'Women': 29,'Arts & Culture': 30,'Crime': 31,'Money': 32,'Weddings': 33,'Arts': 34,'Worldpost': 35, 'The WorldPost': 36,'Education': 37,'College': 38, 'Good news': 39,'Fifty': 40, 'Environment': 41
}
original_to_new_category = {
'Parents': 'LIFESTYLE','Wellness': 'LIFESTYLE','Comedy': 'CULTURE','Parenting': 'LIFESTYLE','Politics': 'POLITICS','Black Voices': 'POLITICS','Queer Voices': 'POLITICS','Entertainment': 'CULTURE','Culture & Arts': 'CULTURE','Tech': 'TECH','Religion': 'CULTURE', 'Style & Beauty': 'LIFESTYLE', 'Healthy living': 'LIFESTYLE','Travel': 'LIFESTYLE','Green': 'ENVIRONMENT','Impact': 'NEWS & WORLD','Business': 'ECONOMY','Divorce': 'LIFESTYLE','Science': 'SCIENCE','Sports': 'SPORTS','Latino Voices': 'POLITICS','World news': 'NEWS & WORLD','Home & Living': 'LIFESTYLE','Media': 'CULTURE','U.S. news': 'NEWS & WORLD','Taste': 'LIFESTYLE','Food & Drink': 'LIFESTYLE','Weird News': 'NEWS & WORLD','Style': 'LIFESTYLE','Women': 'CULTURE','Arts & Culture': 'CULTURE','Crime': 'NEWS & WORLD','Money': 'ECONOMY','Weddings': 'LIFESTYLE','Arts': 'CULTURE','Worldpost': 'NEWS & WORLD', 'The WorldPost': 'NEWS & WORLD','Education': 'EDUCATION','College': 'EDUCATION', 'Good news': 'NEWS & WORLD','Fifty': 'LIFESTYLE', 'Environment': 'ENVIRONMENT'
}
index_to_category = {v: k for k, v in label_dict.items()}
index_to_new_category = {k: original_to_new_category[v] for k, v in index_to_category.items()}

def get_data():
    yesterday = (datetime.now() - timedelta(days=1)).timestamp()
    hot_posts = reddit.subreddit('nottheonion').hot(limit=20)

    data = [
        {
            "title": post.title,
            "score": post.score,
            "created": pd.to_datetime(post.created, unit='s'),
            "url": post.url
        }
        for post in hot_posts if post.created > yesterday
    ]
    return pd.DataFrame(data)

def classify_title(title):
    inputs = tokenizer(title, return_tensors="pt")
    outputs = model(**inputs)
    predicted_category_index = torch.argmax(outputs.logits, dim=-1).item()
    predicted_category_original = index_to_category[predicted_category_index]
    predicted_category_new = original_to_new_category[predicted_category_original]
    return predicted_category_new

def predict_sentiment(text):
    result = nlp(text)[0]
    return result['label']

def generate_prompt(title):
    response = openai.Completion.create(
      engine="text-davinci-003", 
      prompt=f"""
        Instructions:
        Concept: "{title}". /imagine prompt: Start by interpreting the given "{title}". Create a scene that encapsulates the absurdity, humor, or irony contained within, but do not repeat the "{title}" verbatim. Convert your understanding of the context into a straightforward description using simple language.
        You should imagine this scene as a photograph taken by renowned photographer Steve McCurry with a 50mm lens. This means you aim for an ultra-realistic, ultra-detailed 8K resolution image. Avoid figurative language and complex phrases. Include all key elements from the "{title}", like specific characters or locations, using clear and concise language.
        Reflect the time period implied by the "{title}" in your image. For a touch of humor, include a character - a Garfield-like cat observing the scene, reacting with amusement or confusion.
        Describe critical elements like lighting, color temperature, and facial expressions succinctly within four to five lines. Do not include any text in the image.
        Finally, your prompt must conclude with the phrase "Hyperrealistic, ultra-detailed, 8K resolution --v 5.1 --stylize 1000 --ar 6:5". The prompt must also start with the phrase "/imagine prompt:".
        """,
      temperature=0.3,
      max_tokens=200
    )
    return response.choices[0].text.strip()

def generate_hashtags(title):
    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=f"Given the title \"{title}\", generate 30 relevant Instagram hashtags.",
      temperature=0.5,
      max_tokens=60
    )
    hashtags = response.choices[0].text.strip().split(", ")
    hashtags = [tag.replace(" ", "") for tag in hashtags]
    return hashtags

def add_category_and_sentiment(df):
    df['category'] = df['title'].apply(classify_title)
    df['bert_sentiment'] = df['title'].apply(predict_sentiment)
    return df.sort_values(by=['score'], ascending=False)

def add_prompt_and_hashtags(df):
    df['prompt'] = df['title'].apply(generate_prompt)
    df['hashtags'] = df['title'].apply(generate_hashtags)
    return df

# Get the data, classify it, and add category, sentiment, prompt, and hashtags
df = get_data()
df = add_category_and_sentiment(df)
df = df.head(3)  # Keep only the first 3 rows
df = add_prompt_and_hashtags(df)

# Discord setup
intents = discord.Intents.all()
client = discord.Client(intents=intents)
url_received = ""  # Global variable to store the received URL

@client.event
async def on_ready():
    print('Logged in as {0}'.format(client.user))
    channel = client.get_channel(1108705710872739924)  # Use your channel id

    for idx, row in df.iterrows():
        message = f"Title: {row['title']}\n" \
                   f"Score: {row['score']}\n" \
                   f"URL: {row['url']}\n" \
                   f"Category: {row['category']}\n" \
                   f"Sentiment: {row['bert_sentiment']}\n" \
                   f"Prompt: {row['prompt']}\n"
        await channel.send(message)

@client.event
async def on_message(message):
    if message.author.id == client.user.id:
        return

    if 'http' in message.content:
        global url_received
        url_received = message.content
        print(f'Received URL: {url_received}')
        await client.close()

nest_asyncio.apply()
client.run(credentials['DISCORD_TOKEN'])


All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

2023-05-25 10:02:12 INFO     discord.client logging in using static token
2023-05-25 10:02:12 INFO     discord.client logging in using stati

Logged in as Purrposterous#8425
Received URL: https://raw.githubusercontent.com/ralphmartynward/ironhack_09_final-project/main/images/newspurr.png


In [8]:
print(url_received)

https://raw.githubusercontent.com/ralphmartynward/ironhack_09_final-project/main/images/newspurr.png


# Get Midjourney image & transform

In [9]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
import requests
from io import BytesIO
from instabot import Bot

# Create the caption
df['hashtags'] = df['title'].apply(generate_hashtags)
df['caption'] = "🐈 Full story here 👇 " + df['url'] + " " + df['hashtags'].apply(lambda x: ' '.join(x))

# Download the image
response = requests.get(url_received)
image = Image.open(BytesIO(response.content))

# Define the banner text
text = df['title'].iloc[0]  # replace this with your caption

# Select a font
font_path = "/content/drive/My Drive/Admin/School/Online/IronHack/Cours/09 Final Project/DejaVuSans-Bold.ttf"  

# Create the banner image (white background)
banner_height = image.height // 5  # Adjust the height as needed
banner = Image.new('RGB', (image.width, banner_height), "white")

# Create a drawing object for the banner
draw = ImageDraw.Draw(banner)

# Calculate the maximum width for the text
max_text_width = banner.width - 20  # Deducting a margin

# Initial font size (will be adjusted later)
font_size = banner_height // 5  # Adjust this as needed

# Wrap the text initially
wrapper = textwrap.TextWrapper(width=1)  # Initial width (will be adjusted later)
wrapped_text = wrapper.fill(text)

# Create the font object (initially)
font = ImageFont.truetype(font_path, font_size)

# Calculate the width of the wrapped text (initially)
text_width, text_height = draw.textsize(wrapped_text, font=font)

# Decrease the font size until the text fits into the desired width
while text_height > banner_height:
    font_size -= 1
    font = ImageFont.truetype(font_path, font_size)
    wrapper.width = banner.width // font_size
    wrapped_text = wrapper.fill(text)
    text_width, text_height = draw.textsize(wrapped_text, font=font)

# Calculate the position for the centered text
text_x = (banner.width - text_width) // 2
text_y = (banner.height - text_height) // 2

# Draw the wrapped text on the banner
draw.multiline_text((text_x, text_y), wrapped_text, fill="black", font=font)

# Load and place the emoji image
emoji_image = Image.open('/content/drive/My Drive/Admin/School/Online/IronHack/Cours/09 Final Project/images/facepalm.png')
emoji_size = banner_height // 2  # You may need to adjust the size
emoji_margin = banner_height // 6
emoji_image = emoji_image.resize((emoji_size, emoji_size)) 
banner.paste(emoji_image, (banner.width - emoji_image.width - emoji_margin, banner.height - emoji_image.height - emoji_margin))

# Append the banner to the bottom of the image
output_image = Image.new('RGB', (image.width, image.height + banner.height))
output_image.paste(image, (0, 0))
output_image.paste(banner, (0, image.height))

# Initialize a counter for images
image_counter = 1

# Save the output image with a dynamic filename
output_image.save(f"output_image{image_counter}.jpg")
image_counter += 1  # increment the counter

# ----------------------------------------------------------------------------------------------


# Post to IG

In [11]:
import time
from instabot import Bot

# Load the credentials
credentials = {}
with open("cred.txt", "r") as file:
    for line in file:
        key, value = line.strip().split('=')
        credentials[key] = value.strip('\"')

# Retrieve the Instagram credentials from the dictionary
ig_username = credentials["ig_username"]
ig_password = credentials["ig_password"]

caption = df.loc[0, "caption"]

# Initialize and login to the bot
bot = Bot()
bot.login(username=ig_username, password=ig_password)

# Initialize a counter for images
image_counter = 1

# Upload a picture with a dynamic filename
while True:
    try:
        bot.upload_photo(f"output_image{image_counter}.jpg", caption=caption)
        break
    except Exception as e:
        if str(e) == "429":
            print("Too many requests. Sleeping for 5 minutes.")
            time.sleep(300)  # Sleep for 5 minutes (300 seconds)
        else:
            print("Error:", e)
            break

image_counter += 1  # increment the counter


2023-05-25 15:03:31,928 - INFO - Instabot version: 0.117.0 Started
2023-05-25 15:03:31,928 - INFO - Instabot version: 0.117.0 Started
INFO:instabot version: 0.117.0:Instabot version: 0.117.0 Started
DEBUG:instabot version: 0.117.0:Bot imported from /usr/local/lib/python3.10/dist-packages/instabot/bot/bot.py
2023-05-25 15:03:31,947 - INFO - Not yet logged in starting: PRE-LOGIN FLOW!
2023-05-25 15:03:31,947 - INFO - Not yet logged in starting: PRE-LOGIN FLOW!
INFO:instabot version: 0.117.0:Not yet logged in starting: PRE-LOGIN FLOW!
DEBUG:instabot version: 0.117.0:POST to endpoint: accounts/get_prefill_candidates/ returned response: <Response [429]>
DEBUG:instabot version: 0.117.0:Responsecode indicates error; response content: b'{"message":"Please wait a few minutes before you try again.","status":"fail"}'
2023-05-25 15:03:32,519 - ERROR - Request returns 429 error!
2023-05-25 15:03:32,519 - ERROR - Request returns 429 error!
ERROR:instabot version: 0.117.0:Request returns 429 error!
2

KeyboardInterrupt: ignored